In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
image_url = "https://media.springernature.com/lw703/springer-static/image/art%3A10.1038%2F528452a/MediaObjects/41586_2015_Article_BF528452a_Figg_HTML.jpg"

import base64
import httpx
from pathlib import Path

def encode_image(image_source):
    """
    Encode an image from either a URL or local file path to base64.

    Args:
        image_source (str): Either a URL starting with 'http'/'https' or a local file path

    Returns:
        str: Base64 encoded image data
    """
    try:
        # Check if the source is a URL
        if image_source.lower().startswith(('http://', 'https://')):
            # Handle web image
            image_data = base64.b64encode(httpx.get(image_source).content).decode('utf-8')
        else:
            # Handle local file
            path = Path(image_source)
            if not path.exists():
                raise FileNotFoundError(f"Image file not found: {image_source}")

            with open(path, 'rb') as image_file:
                image_data = base64.b64encode(image_file.read()).decode('utf-8')

        return image_data

    except Exception as e:
        raise Exception(f"Error encoding image: {str(e)}")

image_data = encode_image(image_url)

In [ ]:
import os
from groq import Groq

os.environ["GROQ_API_KEY"] = "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"

client = Groq()
llava_model = 'llava-v1.5-7b-4096-preview'
llama31_model = 'llama-3.1-70b-versatile'

## Code copied with gratitude from: https://mer.vin/2024/09/groq-multi-modal/
def image_to_text(client, model, base64_image, prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        model=model
    )

    return chat_completion.choices[0].message.content

prompt = "Describe the image"
image_description = image_to_text(client=client, model=llava_model, base64_image=image_data, prompt=prompt)
print(image_description)

The image features two giant lizards or alligators laying on the grass in a grassy area. These large animals appear to be engaging in a wrestling match, creating an exciting scene. In the background, there is a body of water, which appears to be the ocean. The lizards are positioned close to each other, capturing the intense struggle between the two creatures.


In [ ]:
## Code copied with gratitude from: https://mer.vin/2024/09/groq-multi-modal/
def short_story_generation(client, image_description, topic):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"You are a children's book author. Write a short story about the scene depicted in this image or images. The story is about Topic - {topic}",
            },
            {
                "role": "user",
                "content": image_description,
            }
        ],
        model=llama31_model
    )

    return chat_completion.choices[0].message.content

prompt = '''
Describe this image in detail, including the appearance of the people and any notable actions or behaviors.
'''
image_description = image_to_text(client, llava_model, image_data, prompt)

topic = "dad going away for work"

print("\n\n--- Image Description (Labradoodle) ---\n")
print(image_description)

print("\n\n--- Short Story (Based on Labradoodle) ---\n")
print(short_story_generation(client, image_description, topic))



--- Image Description (Labradoodle) ---

The image displays a group of hilarious comedic lizards in an open field, seeming to have a playful interaction as if engaged in an ongoing wrestling match. There are two main lizards, one on top of the other, creating an interesting and captivating scene. Two people can be seen close to the lizards, observing the entertaining spectacle. It appears to be a sunny day, and the lizards are capturing the attention of the nearby individuals.


--- Short Story (Based on Labradoodle) ---

That image doesn't match the scenario of 'Dad going away for work'. 

However, I would be happy to create a short story based on that topic for you. Here's one:

---

"Benny's Big Goodbye"

Benny the little lizard lived in a cozy rock with his family in the open fields. He loved playing with his siblings and exploring the sunshine-kissed grasslands. But one day, his dad had to say goodbye.

"Daddy, why do you have a suitcase?" Benny asked, his curious eyes wide with

In [ ]:
first_image = "https://plus.unsplash.com/premium_photo-1664474619075-644dd191935f?fm=jpg&q=60&w=3000&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8aW1hZ2V8ZW58MHx8MHx8fDA%3D"
second_image = "https://cdn.pixabay.com/photo/2018/08/14/13/23/ocean-3605547_640.jpg"
third_image = "https://img.pikbest.com/png-images/20240718/lord-krishna-vector-design-image_10670443.png!w700wp"

image_data1 = encode_image(first_image)
image_data2 = encode_image(second_image)
image_data3 = encode_image(third_image)

image_description1 = image_to_text(client, llava_model, image_data1, prompt)
image_description2 = image_to_text(client, llava_model, image_data2, prompt)
image_description3 = image_to_text(client, llava_model, image_data3, prompt)

total_description = image_description1 + "\n\n" + image_description2 + "\n\n" + image_description3
print(total_description)

print("\n\n--- Short Story (Based on combined description) ---\n")
print(short_story_generation(client, total_description, topic=topic))


The image features a young woman wearing a hat, standing next to a street sidewalk. She's holding two different cameras, taking a photo aiming towards the left side. Her focus is likely on capturing a captivating picture. 

The scene also includes a couple of people in the background, one of them wearing a hat and another carrying a handbag. The area appears to be busy with pedestrians, which further emphasizes the street's bustling atmosphere.

A small sailboat with two people on board is floating across a calm lake under a beautiful night sky during two different time frames. In the wilderness, the star is beginning to set, while the ship still rides the water. In the second frame, the sky is full of stars, and they are changing under a sky that lights up, highlighting their silhouetted forms. Each person on the boat appears to be playing with sparklers on the night, adding to the allure of the scene. The boat's location is on top of the large body of water at night, with a few cloud